In [244]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [245]:
data = pd.read_csv("dataset.csv")

In [246]:
data.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [247]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [248]:
# Clean TotalCharges where empty value

data.loc[data["tenure"] == 0, "TotalCharges"] = data.loc[data["tenure"] == 0, "tenure"] * data.loc[data["tenure"] == 0, "MonthlyCharges"]
data["TotalCharges"] = pd.to_numeric(data["TotalCharges"])

In [250]:
for i in data.columns:
    if data[i].dtype == "object":
        print(f"{i}: {data[i].unique()}")

customerID: ['7590-VHVEG' '5575-GNVDE' '3668-QPYBK' ... '4801-JZAZL' '8361-LTMKD'
 '3186-AJIEK']
gender: ['Female' 'Male']
Partner: ['Yes' 'No']
Dependents: ['No' 'Yes']
PhoneService: ['No' 'Yes']
MultipleLines: ['No phone service' 'No' 'Yes']
InternetService: ['DSL' 'Fiber optic' 'No']
OnlineSecurity: ['No' 'Yes' 'No internet service']
OnlineBackup: ['Yes' 'No' 'No internet service']
DeviceProtection: ['No' 'Yes' 'No internet service']
TechSupport: ['No' 'Yes' 'No internet service']
StreamingTV: ['No' 'Yes' 'No internet service']
StreamingMovies: ['No' 'Yes' 'No internet service']
Contract: ['Month-to-month' 'One year' 'Two year']
PaperlessBilling: ['Yes' 'No']
PaymentMethod: ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
Churn: ['No' 'Yes']


In [251]:
# Make all categorical data numerical

data["gender"] = (data["gender"].str.lower() == "male").astype(int)
data["Partner"] = (data["Partner"].str.lower() == "yes").astype(int)
data["Dependents"] = (data["Dependents"].str.lower() == "yes").astype(int)
data["PhoneService"] = (data["PhoneService"].str.lower() == "yes").astype(int)
data["MultipleLines"]  = data["MultipleLines"].str.lower().map({"no phone service": 0, "yes": 1, "no": 2})
data["InternetService"]  = data["InternetService"].str.lower().map({"dsl": 0, "fiber optic": 1, "no": 2})
data["OnlineSecurity"]  = data["OnlineSecurity"].str.lower().map({"no internet service": 0, "yes": 1, "no": 2})
data["OnlineBackup"]  = data["OnlineBackup"].str.lower().map({"no internet service": 0, "yes": 1, "no": 2})
data["DeviceProtection"]  = data["DeviceProtection"].str.lower().map({"no internet service": 0, "yes": 1, "no": 2})
data["TechSupport"]  = data["TechSupport"].str.lower().map({"no internet service": 0, "yes": 1, "no": 2})
data["StreamingTV"]  = data["StreamingTV"].str.lower().map({"no internet service": 0, "yes": 1, "no": 2})
data["StreamingMovies"]  = data["StreamingMovies"].str.lower().map({"no internet service": 0, "yes": 1, "no": 2})
data["Contract"] = data["Contract"].str.lower().map({"month-to-month": 0, "one year": 1, "two year": 2})
data["PaperlessBilling"] = (data["PaperlessBilling"].str.lower() == "yes").astype(int)
data["PaymentMethod"] = data["PaymentMethod"].str.lower().map({"electronic check": 0, "mailed check": 1, "bank transfer (automatic)": 2, "credit card (automatic)": 3})

data["Churn"] = (data["Churn"].str.lower() == "yes").astype(int)

In [252]:
# drop customerID as it doesn't influence churn
data.drop(columns=["customerID"], inplace=True)

In [253]:
data["tenure"] = (data["tenure"] - data["tenure"].mean()) / data["tenure"].std()
data["MonthlyCharges"] = (data["MonthlyCharges"] - data["MonthlyCharges"].mean()) / data["MonthlyCharges"].std()
data["TotalCharges"] = (data["TotalCharges"] - data["TotalCharges"].mean()) / data["TotalCharges"].std()

In [254]:
y = data["Churn"]
X = data.drop(columns=["Churn"], inplace=False)

In [255]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [257]:
model_params = [
    ("Logistic Regression", LogisticRegression(), {
        "C": [0.01, 0.1,1, 10],
        "penalty": ["l2"],
        "solver": ["lbfgs"],
        "max_iter": [10000]
    }),
    ("Random Forest", RandomForestClassifier(), {
        "n_estimators": [100, 200],
        "max_depth": [3, 4, 5, None]
    }),
    ("SVC", SVC(), {
        "C": [0.1, 1, 10],
        "kernel": ["linear", "rbf"],
        "gamma": ["scale", "auto"]
    }),
]

best_models = {}

for name, model, params in model_params:
    grid = GridSearchCV(model, param_grid=params, cv=5, scoring="accuracy")
    grid.fit(X_train, y_train)
    print(f"{name} | Best Params: {grid.best_params_} | Best Score: {grid.best_score_:.4f}")
    best_models[name] = grid.best_estimator_

Logistic Regression | Best Params: {'C': 0.1, 'max_iter': 10000, 'penalty': 'l2', 'solver': 'lbfgs'} | Best Score: 0.7998
Random Forest | Best Params: {'max_depth': 5, 'n_estimators': 200} | Best Score: 0.7962
SVC | Best Params: {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'} | Best Score: 0.7989


In [260]:
for name, model in best_models.items():
    y_pred = model.predict(X_test)
    print(f"{name} | Test Accuracy: {accuracy_score(y_test, y_pred):.4f}")

Logistic Regression | Test Accuracy: 0.8204
Random Forest | Test Accuracy: 0.8006
SVC | Test Accuracy: 0.8162
